### Exercício 4

Tomando em conta o motor turbo hélice com as seguintes características operacionais no início da decolagem (Mach = 0 e rotação máxima) ao nível do mar (Ta = 288,2 K e Pa = 101,3 kPa): vazão de ar 6,7 kg/s, razão de compressão 12, temperatura total na saída da câmara de combustão 1400 K, além de :

<div><center>
<img src="images/questao4_tabela.png" alt="Tabela da questão 4" style="width: 300px;"/>
</center></div>

Poder calorífico inferior do combustível: 4500 kJ/kg
Cp turbina livre isentrópica: 1.16 kJ/(kg.K)
Cp turbina livre: 1.11 kJ/(kg.K)

In [36]:
import model_reference as mf
import numpy as np
import pandas as pd
import warnings

values = {
    'mass_flow': 6.7,  # Vazão mássica = vazão de ar no compressor [kg/s]
    't04': 1400,       # Temperatura da saída da câmara de combustão [K]
    'prc': 12,         # Razão de pressão do compressor
    'mach': 0,         # Mach de voo [M]
    'ta': 288.2,       # Temperatura ambiente [K]
    'pa': 101.3,       # Pressão ambiente [kPa]
    'n_d': 0.85,       # Eficiência da entrada de ar
    'gamma_d': 1.4,    # Gamma da entrada de ar
    'n_c': 0.75,       # Eficiência do compressor
    'gamma_c': 1.37,   # Gamma do compressor
    'n_b': 1,          # Eficiência da câmera de combustão
    'gamma_b': 1.35,   # Gamma da câmera de combustão
    'n_t': 0.9,        # Eficiência da turbina do compressor
    'gamma_t': 1.33,   # Gamma da turbina do compressor
    'n_tl': 0.9,       # Eficiência da turbina livre
    'gamma_tl': 1.33,  # Gamma da turbina livre
    'n_n': 0.98,       # Eficiência do bocal de gases quentes
    'gamma_n': 1.36,   # Gamma do bocal de gases quentes
    'pc_fuel': 45000,  # Poder calorífico inferior do combustível [kJ/kg]
    'cp_fuel': 1.110,  # Cp no combustor [kJ/(kg.K)]
    'cp_tl': 1.160,    # Cp da turbina livre [kJ/(kg.K)]
    'r': 288.3,        # R médio m2/(s2.K)
    'pr_tl': 2.87,                # Razão de pressão da turbina livre    
    'gearbox_power_ratio': 0.98,     # Razão de potência da gearbox
    'propeller_efficiency': 0.85, # Eficiência da hélice
    }

# Como os valores não foram dados, considera-se que a razão de potência da gearbox 
# é 0.98 e a eficiência da hélice é 0.85. Também, como não foram informados alguns outros
# valores foram utilizados: n_b = 1, gamma_b = 1.35, cp_fuel = 1.11, R = 288.3

# Para calibrar o motor inicialmente para permitir que a potência do motor produzida seja 1440 kW
turbine_power_target = 1440 #[kW]

turbine_pressure_ratio = 2.0
values["pr_tl"] = turbine_pressure_ratio
turbine_pressure_ratio_delta = 0.0001

turboprop = mf.TurboProp(values)
turboprop.set_n2(1)

performance_results = turboprop.sumarise_results()

while(int(performance_results.loc['turbine_power']) < turbine_power_target):
    turbine_pressure_ratio = turbine_pressure_ratio + turbine_pressure_ratio_delta
    values["pr_tl"] = turbine_pressure_ratio
    turboprop = mf.TurboProp(values)
    performance_results = turboprop.sumarise_results()

turbine_pressure_ratio = turbine_pressure_ratio - turbine_pressure_ratio_delta
values["pr_tl"] = turbine_pressure_ratio
turboprop = mf.TurboProp(values)
performance_results = turboprop.sumarise_results()
print(int(performance_results.loc['turbine_power']))
print(turbine_pressure_ratio)
performance_results

# Calibrar o motor para potência 1282kW
N2 = 1
N2_delta = 0.0001
turbine_power_target = 1282 #[kw]

while(int(performance_results.loc['turbine_power']) > turbine_power_target):
    N2 = N2 - N2_delta
    turboprop = mf.TurboProp(values)
    turboprop.set_n2(N2)
    performance_results = turboprop.sumarise_results()

print(int(performance_results.loc['turbine_power']))
print(N2)
performance_results
print("T04: ",turboprop.sumarise().loc['t04'])


1439
2.406000000000857
1282
0.9572000000000047
T04:  0.9572    1335.134755
Name: t04, dtype: float64
